In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# assign the met database to a variable name
db = mongo['met']

# assign the collection to a variable
artifacts = db['artifacts']

In [20]:
# Build the aggregation pipeline
# Write a match query to find only the documents about artifacts that have a width greater than or equal to 40cm.
match_query = {'$match': {'measurements.elementMeasurements.Width': {'$gte': 40}}}

# Write an aggregation query that counts the number of documents, grouped by "country"
group_query = {'$group': {'_id': "$country", 'count': { '$sum': 1 }}}

# Create a dictionary that will allow the pipeline to sort by count in descending order
sort_values = {'$sort': { 'count': -1 }}

match_query2 = {'$match': {'_id': {'$eq': 'Peru'}}}

# Put the pipeline together
pipeline = [match_query, group_query,match_query2, sort_values]
print(pipeline)

[{'$match': {'measurements.elementMeasurements.Width': {'$gte': 40}}}, {'$group': {'_id': '$country', 'count': {'$sum': 1}}}, {'$match': {'_id': {'$eq': 'Peru'}}}, {'$sort': {'count': -1}}]


In [16]:
# Run the pipeline through the aggregate method and save the results to a variable
results = list(artifacts.aggregate(pipeline))
results

[{'_id': 'Peru', 'count': 13}]

In [17]:
# Print the number of countries in the result
print("Number of countries in result: ", len(results))

Number of countries in result:  1


In [18]:
# Print the first 10 results
pprint(results[0:10])

[{'_id': 'Peru', 'count': 13}]


In [19]:
# Convert mongo result to Pandas DataFrame
result_df = pd.DataFrame(results)

print("Rows in DataFrame: ", len(result_df))
result_df.head(10)

Rows in DataFrame:  1


,_id,count
0,Peru,13


Data Source: [The Metropolitan Museum of Art](https://www.metmuseum.org/) (2022). The Metropolitan Museum of Art Collection API https://metmuseum.github.io/. Licenced under the [Creative Commons 0 Licence](https://creativecommons.org/publicdomain/zero/1.0/).<br />
Accessed Oct 3, 2022. Data collected from departmentId=5 ("Arts of Africa, Oceania, and the Americas") and search string "animal".